*Copyright (C) 2023 Intel Corporation*<br>
*SPDX-License-Identifier: BSD-3-Clause*<br>
*See: https://spdx.org/licenses/*

---

# Satellite Scheduler Tech Demo
This notebook demonstrates the usage of a Lava-Optimization QUBO solver to schedule requests for a constellation of Earth Observation satellites.
The notebook uses the SatelliteSchedulingProblem API to generate synthetic problem instances, convert the problems into QUBO matrices, and then run
the Lava solver to find a satisfactory schedule.

### Scenario Description
Earth Observation satellites orbit the Earth on fixed trajectories with each orbital pass taking between 30 minutes and a few hours. During an orbit,
the satellite can reorient itself to observe different positions on the Earth's surface with its sensors. The ability to reorient is limited by
the satellite's actuators to a maximum rotational rate. For a given satellite to satisfy two sequential observation requests, there must be adequate
time between the requests for the satellite to reorient without exceeding its maximum rotational rate. For simple orbits, the time between requests
is essentially determined by the difference in longitude coordinates of the two requests, divided by the longitudinal velocity of the satellite.

<center><img src="SatSchDemoSchematic.png" width="500"/></center>

### Solution Strategy
The physical constraints of the satellite scheduling problem can be mapped into QUBO by creating a graph corresponding to the vehicles and requests
that are currently being scheduled. Any two requests which cannot be observed by the same vehicle will be connected in the graph by an edge,
indicating a hard constraint between those requests. Using a QUBO solver, we can then find a Maximal Independent Set of the graph, corresponding
to a locally-optimal schedule. This method of scheduling requests for a constellation of satellites can scale up to create schedules for larger
constellations receiving more requests than a conventional scheduling algorithm.

In [ ]:
import os
import numpy as np
from matplotlib import pyplot as plt
from lava.lib.optimization.apps.scheduler.problems import SatelliteScheduleProblem
from lava.lib.optimization.apps.scheduler.solver import SatelliteScheduler

#### Create a SchedulingProblem object

In [ ]:
scheduling_problem = SatelliteScheduleProblem(num_satellites=9,
                                              num_requests=250)
scheduling_problem.view_height = 0.50
scheduling_problem.turn_rate = 5.0
scheduling_problem.generate(0)
scheduling_problem.plot_problem()

#### Create a Scheduler object
Scheduler consumes a SchedulingProblem along with QUBO specific parameters

In [ ]:
scheduler = SatelliteScheduler(scheduling_problem,
                               qubo_weights=(1, 8),
                               probe_loihi_exec_time=False)

##### Solve using NetworkX module

In [ ]:
scheduler.solve_with_netx()
print(f'Scheduled {scheduler.netx_solution.shape[0]} Requests.')

##### Solve using Lava QUBO Solver on Loihi 2

In [ ]:
# os.environ["LOIHI_GEN"]="N3B3"
# os.environ["PARTITION"]="kp_dev"
# os.environ["SLURM"]="1"

In [ ]:
scheduler.qubo_hyperparams = ({"temperature": 1},
                              True)
scheduler.lava_backend = "CPU"
scheduler.solve_with_lava_qubo(timeout=1000)

In [ ]:
scheduler.plot_solutions()

In [ ]:
scheduler.lava_solution.shape[0]

In [ ]:
GSS = {
    "view_height": [0.1, 0.2, 0.3],
    "turn_rate": [1.0, 3.0, 5.0, 7.0],
    "num_satellites": [2, 5, 7, 9],
    "qubo_weights": [[1, 4], [1, 8], [2, 8], [2, 16], [4, 16]]
}

def run_ssp(arg1: int, arg2: int, arg3: int, arg4: int, arg5: int, num_repeats: int = 5, num_requests: int = 1000, plotting: bool = False) -> float:

    # view_height = GSS["view_height"][int(arg1)]
    # turn_rate = GSS["turn_rate"][int(arg2)]
    view_height = float(arg1)
    turn_rate = float(arg2)
    num_satellites = int(arg3)
    qubo_weight_0 = float(arg4)
    qubo_weight_1 = qubo_weight_0 * float(arg5)

    request_logs = []
    for repeat_idx in range(num_repeats):
        scheduling_problem = SatelliteScheduleProblem(
            num_satellites=num_satellites,
            num_requests=num_requests
        )
        scheduling_problem.view_height = view_height
        scheduling_problem.turn_rate = turn_rate
        scheduling_problem.generate(repeat_idx)

        if plotting:
            secheduling_problem.plot_problem()

        scheduler = SatelliteScheduler(
            scheduling_problem,
            qubo_weights=(qubo_weight_0, qubo_weight_1),
            # probe_loihi_exec_time=True
        )

        scheduler.qubo_hyperparams = ({"temperature": 1}, True)
        scheduler.lava_backend = "CPU"
        scheduler.solve_with_lava_qubo(timeout=1000)

        num_satisfies = scheduler.lava_solution.shape[0]
        request_logs.append(num_satisfies)


    avg_requests = np.mean(request_logs)
    avg_requests = -1 * avg_requests
    return avg_requests
        

In [ ]:
results = run_ssp(1, 1, 2, 1, 2.1)

In [ ]:
results

In [ ]:
# -------------
# Grid Search
# -------------


In [ ]:
# ------
# BO
# ------
from skopt import Space
from skopt.space import Real, Integer
import time

search_space_int = Space([
    Real(0.001, 10,  name="view_height"),
    Real(0.001, 10, name="turn rate"),
    Integer(2, 6,   name="number of satellites"),
    Real(1, 10,     name="qubo_weight_0"),
    Real(2.1, 10,   name="qubo_weight_1_scaler")
])

num_initial_points = 50
num_iterations = 150
optimizer_class = "gp-cpu"

from omegaconf import DictConfig
from lmao.solver import BOSolver

optimizer_config = DictConfig({
    "num_initial_points": num_initial_points,
    "max_iterations": num_iterations,
    "num_processes": 1,
    "num_repeats": 1,
    "optimizer_class": optimizer_class,
    "optimizer": {
        "num_initial_points": num_initial_points,
    },
    "seed": 10,
})

solver = BOSolver(optimizer_config)

start_time = time.time()
result = solver.solve(
    run_ssp,
    use_lp=False,
    search_space=search_space_int,
)
total_time = time.time() - start_time
print(f"Total Time: {total_time}")

In [ ]:
result

In [ ]:
plt.plot(result["y_log"])